# Анализ детерминированного шума

In [2]:
import numpy as np
import pandas as pd
from plotly.offline import plot
import itertools

###### Импорт excel

In [3]:
path_file = ''
lists_excel = ['1-1', '1-2', '1-3', '2-1', '2-2', '2-3', '3-1', '3-2', '3-3']
df = pd.read_excel(path_file
                   , lists_excel
                   , usecols=['Частота(Гц)', 'Звуковое давление(Па)']
                   , index_col='Частота(Гц)')
p0 = 2 * 10 ** (-5)
for i in df:
    df['%s' % i].columns = ['pressure']
    df['%s' % i]['Level_pressure'] = 20 * np.log10(df['%s' % i]['pressure'] / p0)

###### Функции  чтения и записи файла с пиками

In [4]:
# Запись в файл
def write(pos):
    with open('indexes_%s.txt' % pos, 'w') as f:
        for i in indexes:
            f.write(str(i) + '\n')

    with open('values_%s.txt' % pos, 'w') as f:
        for i in values:
            f.write(str(i) + '\n')


# чтение файла
def read(pos):
    with open('indexes_%s.txt' % pos, 'r') as f:
        indexes = [int(line) for line in f]

    with open('values_%s.txt' % pos, 'r') as f:
        values = [float(line) for line in f]

    poss = np.array([indexes, values]).transpose()
    return poss

##### Импорт найденных экстремумов

Экстремумы были предварительно найдены вручную и записаны в текстовые файлы indexes_%.txt и values_%.txt где %-номер положения. Здесь же в кортеж peaks мы считываем эти txt файлы с индексами и значениями экстремумов.

In [5]:
peaks = {'%s' % i: read(i) for i in lists_excel}

##### Генерация аналитических частот

df-mods - датафрейм с аналитическими модами

In [6]:
w = 345
a = 0.15
b = 0.25
c = 0.15
itertool = list(itertools.product([0, 1, 2, 3], repeat=3))
df_mods = pd.DataFrame(itertool, columns=['l', 'm', 'n'])
df_mods['freq'] = np.round(
    (w / 2) * np.sqrt((df_mods['l'] / a) ** 2 + (df_mods['m'] / b) ** 2 + (df_mods['n'] / c) ** 2), 0)
df_mods = df_mods.sort_values(by='freq').reset_index(drop=True)
df_mods = df_mods.drop_duplicates(subset=['freq'], keep='last').reset_index(drop=True)
df_mods = df_mods.drop([0, 38, 39], axis=0)

Описание:
find_best(df_mods, peaks[name], 2) - функция поиска наиболее близкой аналитический частоты к экспериментальной. 

Входные данные:
    - датафрейм с аналит.частотами (df_mods);
    - данные с экстремумами для определенного положения peaks[name] где name - положение ('1-1'...);
    - порог определения (в данном случае 2%)

В результате имеем:
    - best_mas - список отобранных частот
    - best_mas_all - список всех частот
    - not_best_mas - список частот, которые остались не определенными

In [8]:
def find_best(df_mods, poss, threshold):
    # функция поиска наиболее близких аналит.частот
    global best_mas
    global best_mas_all
    global not_best_mas
    best_mas = np.array([])

    for i in poss[:, 0]:
        mas = np.array([])
        for j in df_mods.values:
            delta = np.abs(i - j[3])
            mas = np.append(mas, [j[3], i, delta])
        mas = mas.reshape(37, 3)
        best_analit, exper = mas[np.argmin(mas[:, 2]), 0], mas[np.argmin(mas[:, 2]), 1]
        delta_percent = 100 * np.abs(1 - (best_analit / exper))
        best_mas = np.append(best_mas, [float(best_analit), float(exper), round(delta_percent, 1)])

    best_mas = best_mas.reshape(len(poss[:, 0]), 3)
    best_mas_all = best_mas
    k = np.where(best_mas[:, 2] > threshold)  # Находим строки, где разница более порога
    not_best_mas = best_mas[k]
    best_mas = np.delete(best_mas, (k), axis=0)  # удаляем строки, где разница более порога
    unique, unique_idx = np.unique(best_mas[:, 0], return_index=True)  # ищем только уникальные аналит.частоты
    best_mas = best_mas[unique_idx]  # оставляем только уникальные

##### Отрисовка графика

In [24]:
def plot(name):
    fig = go.Figure()

    fig.add_trace(go.Scatter(name='Уровень звукового давления', line=dict(width=0.5), fill='tozeroy', mode='lines',
                             showlegend=False,
                             x=df[name].index,
                             y=df[name].Level_pressure))

    fig.add_trace(
        go.Scatter(name='Экстремумы подтвержденные<br>аналитически', mode='markers', marker=dict(color='green', size=5),
                   x=peaks[name][:, 0],
                   y=peaks[name][:, 1]))
    fig.add_trace(
        go.Scatter(name='Экстремумы неподтвержденные<br>аналитически', mode='markers', marker=dict(color='red', size=5),
                   x=not_best_mas[:, 1],
                   y=df[name]['Level_pressure'].loc[not_best_mas[:, 1]]))

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgb(220,220,220)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgb(220,220,220)')

    '''    for i in df_mods.values:
        fig.add_shape(dict(type="line",opacity=0.7,
                           line=dict(color="Crimson",width=1),
                           x0=int(i[3]),y0=0,
                           x1=int(i[3]),y1=90))'''
    for i in best_mas:
        fig.add_shape(dict(type="line", opacity=0.8,
                           line=dict(color="Green", width=1.5),
                           x0=int(i[0]), y0=0,
                           x1=int(i[0]), y1=90))

    fig.add_trace(
        go.Scatter(name='Аналитические частоты', line=dict(color='Green', width=1), mode='lines', showlegend=True,
                   x=np.ones(90) * best_mas[0][0],
                   y=np.arange(0, 90, 1)))

    fig.update_layout(width=1000, height=500, plot_bgcolor='rgb(255,255,255)',
                      title=dict(text='Анализ детерминированного шума для положения %s' % name, font=dict(size=17)),
                      xaxis=dict(title='Частота, Гц', tickvals=df[name].index[::400], tickfont=dict(size=14)),
                      yaxis=dict(title='Уровень звукового давления, дБ', tickfont=dict(size=14)),
                      showlegend=True, legend_orientation="h", legend=dict(x=0.025, y=1.1))
    fig.show()

In [16]:
def result(name):
    find_best(df_mods, peaks[name], 2)
    plot(name)

In [37]:
result('1-1')

In [10]:
columns = ['Аналитическая частота, Гц','Экспериментальная частота, Гц','Отличие, %']

In [11]:
df_1_1 = pd.DataFrame(best_mas, columns=columns)
df_1_1_all = pd.DataFrame(best_mas_all, columns= columns)

In [13]:
df_1_2 = pd.DataFrame(best_mas, columns=columns)
df_1_2_all = pd.DataFrame(best_mas_all, columns= columns)

In [15]:
df_1_3 = pd.DataFrame(best_mas, columns=columns)
df_1_3_all = pd.DataFrame(best_mas_all, columns=columns)

In [17]:
df_2_1 = pd.DataFrame(best_mas, columns=columns)
df_2_1_all = pd.DataFrame(best_mas_all, columns=columns)

In [19]:
df_2_2 = pd.DataFrame(best_mas, columns=columns)
df_2_2_all = pd.DataFrame(best_mas_all, columns=columns)

In [21]:
df_2_3 = pd.DataFrame(best_mas, columns=columns)
df_2_3_all = pd.DataFrame(best_mas_all, columns=columns)

In [23]:
df_3_1 = pd.DataFrame(best_mas, columns=columns)
df_3_1_all = pd.DataFrame(best_mas_all, columns=columns)

In [25]:
df_3_2 = pd.DataFrame(best_mas, columns=columns)
df_3_2_all = pd.DataFrame(best_mas_all, columns=columns)

In [27]:
df_3_3 = pd.DataFrame(best_mas, columns=columns)
df_3_3_all = pd.DataFrame(best_mas_all, columns=columns)

In [28]:
with pd.ExcelWriter('results.xlsx') as writer:
    df_1_1.to_excel(writer, encoding='cp1251', sheet_name='1-1', index=False)
    df_1_1_all.to_excel(writer, encoding='cp1251', sheet_name='1-1', startcol=4, index=False)

    df_1_2.to_excel(writer, encoding='cp1251', sheet_name='1-2', index=False)
    df_1_2_all.to_excel(writer, encoding='cp1251', sheet_name='1-2', startcol=4, index=False)

    df_1_3.to_excel(writer, encoding='cp1251', sheet_name='1-3', index=False)
    df_1_3_all.to_excel(writer, encoding='cp1251', sheet_name='1-3', startcol=4, index=False)

    df_2_1.to_excel(writer, encoding='cp1251', sheet_name='2-1', index=False)
    df_2_1_all.to_excel(writer, encoding='cp1251', sheet_name='2-1', startcol=4, index=False)

    df_2_2.to_excel(writer, encoding='cp1251', sheet_name='2-2', index=False)
    df_2_2_all.to_excel(writer, encoding='cp1251', sheet_name='2-2', startcol=4, index=False)

    df_2_3.to_excel(writer, encoding='cp1251', sheet_name='2-3', index=False)
    df_2_3_all.to_excel(writer, encoding='cp1251', sheet_name='2-3', startcol=4, index=False)

    df_3_1.to_excel(writer, encoding='cp1251', sheet_name='3-1', index=False)
    df_3_1_all.to_excel(writer, encoding='cp1251', sheet_name='3-1', startcol=4, index=False)

    df_3_2.to_excel(writer, encoding='cp1251', sheet_name='3-2', index=False)
    df_3_2_all.to_excel(writer, encoding='cp1251', sheet_name='3-2', startcol=4, index=False)

    df_3_3.to_excel(writer, encoding='cp1251', sheet_name='3-3', index=False)
    df_3_3_all.to_excel(writer, encoding='cp1251', sheet_name='3-3', startcol=4, index=False)

##### Постанализ

В данном разделе происходит отрисовка гистограммы с частотами которые считаются определенными и не определенными для каждого из положений. 
- чтение файла из экселя results.xlsx
- отрисовка гистограммы

In [71]:
path_to_result = ''
df_post = pd.read_excel(path_to_result
                        , 'Итог'
                        , usecols=['Индекс', 'Не определено, шт', 'Определено, шт', 'Определено, %']
                        , index_col='Индекс', nrows=9)
df_post['Определено, %'] = 100 * df_post['Определено, %']

In [74]:
fig = go.Figure()
x = df_post.index

fig.add_trace(go.Bar(
    name='Определено',
    x=x,
    y=df_post['Определено, шт']))

fig.add_trace(go.Bar(
    name='Не определено',
    x=x,
    y=df_post['Не определено, шт']))

fig.update_layout(width=1000, height=500, plot_bgcolor='rgb(255,255,255)', barmode='relative',
                  xaxis=dict(tickvals=df_post.index, tickfont=dict(size=14), showgrid=True,
                             gridwidth=1, gridcolor='rgb(220,220,220)'),
                  yaxis=dict(title='Количество', tickfont=dict(size=14), showgrid=True,
                             gridwidth=1, gridcolor='rgb(220,220,220)'),
                  showlegend=True, legend_orientation="h", legend=dict(x=0.025, y=1.1))
fig.show()

##### Пострезульт1

В данном разделе подсчитывается точность определения частот.
Комментарии к выполнению:

1) После выполнения функции result('...') формируется глобальный список best_mas, содержащий в себе массив с тремя столбцами: аналитическая частота, экспериментальная и разница в процентах между ними. Нужно выполнить функцию result('...') для положения, например '1-1' (result('1-1'). В best_mas сохранятся частоты для этого положения.
2) Инициализируем функцию diap() - она делит массив best_mas на 4 части, по точности определения 0-0.5, 0.5-1, 1-1.5, 1.5-2 % и суммирует количество частот в каждой части.
3) записываем в переменную p1 получившийся кортеж и сохраняем
4) возвращаемся к функции result('...') и выполняем ее для положения '1-2' и повторяем шаги.

- p_sum - будет содержать в себе суммы по четырем частям за все положения
- результаты p1,p2...,p_sum так же записаны в эксель, для того, чтобы при повторном обращении к алгоритму не выполняться все действия а просто считать их из экселя

In [155]:
def diap():
    result_diaps = {}
    values = np.array([0, 0.5, 1, 1.5, 2])
    for i, j in enumerate(values):
        if i <= 3:
            if i == 0:
                k = np.where((best_mas[:, 2] >= values[i]) & (best_mas[:, 2] <= values[i + 1]))
            else:
                k = np.where((best_mas[:, 2] > values[i]) & (best_mas[:, 2] <= values[i + 1]))
            best_mas_k = best_mas[k]
            n = len(best_mas_k)
            result_diaps['%s-%s' % (values[i], values[i + 1])] = n
    return result_diaps

Последовательное выполнение операций от p1 до p9

In [156]:
p1 = diap()

In [197]:
p_sum = {}
for i in p1:
    p_sum[i] = p1[i] + p2[i] + p3[i] + p4[i] + p5[i] + p6[i] + p7[i] + p8[i] + p9[i]

запись результатов в файл 'results_pie.xlsx', чтобы не потерялись

In [237]:
columns = ['Количество частот']
p1_df = pd.DataFrame.from_dict(p1, orient='index', columns=columns)
p2_df = pd.DataFrame.from_dict(p2, orient='index', columns=columns)
p3_df = pd.DataFrame.from_dict(p3, orient='index', columns=columns)
p4_df = pd.DataFrame.from_dict(p4, orient='index', columns=columns)
p5_df = pd.DataFrame.from_dict(p5, orient='index', columns=columns)
p6_df = pd.DataFrame.from_dict(p6, orient='index', columns=columns)
p7_df = pd.DataFrame.from_dict(p7, orient='index', columns=columns)
p8_df = pd.DataFrame.from_dict(p8, orient='index', columns=columns)
p9_df = pd.DataFrame.from_dict(p9, orient='index', columns=columns)

In [239]:
with pd.ExcelWriter('results_pie.xlsx') as writer:
    p1_df.to_excel(writer, encoding='cp1251', sheet_name='1-1')

    p2_df.to_excel(writer, encoding='cp1251', sheet_name='1-2')

    p3_df.to_excel(writer, encoding='cp1251', sheet_name='1-3')

    p4_df.to_excel(writer, encoding='cp1251', sheet_name='2-1')

    p5_df.to_excel(writer, encoding='cp1251', sheet_name='2-2')

    p6_df.to_excel(writer, encoding='cp1251', sheet_name='2-3')

    p7_df.to_excel(writer, encoding='cp1251', sheet_name='3-1')

    p8_df.to_excel(writer, encoding='cp1251', sheet_name='3-2')

    p9_df.to_excel(writer, encoding='cp1251', sheet_name='3-3')

    p_sum_df.to_excel(writer, encoding='cp1251', sheet_name='итог')


Отрисовка пирога

In [227]:
labels = [i for i in p_sum.keys()]
values = [i for i in p_sum.values()]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(legend=dict(x=0.75, y=0.5))

fig.show()

##### Алгоритм нахождения экстремумов в заданном диапазоне (требовалось вначале)

In [500]:
values = []
indexes = []


def peaks(list_name, val1, val2, name):
    if name == 'x':
        value = df[list_name]['Level_pressure'].loc[val1:val2].max()
        index = df[list_name]['Level_pressure'].loc[val1:val2].idxmax()
    elif name == 'n':
        value = df[list_name]['Level_pressure'].loc[val1:val2].min()
        index = df[list_name]['Level_pressure'].loc[val1:val2].idxmin()
    values.append(value)
    indexes.append(index)


peaks('3-3', 4810, 4912, 'x')  # обращение к функции

write('3-3')  # обращение к записи получившегося массива пиков в txt файл
